In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import xarray as xr
import datashader as ds
import cartopy.crs as ccrs

import panel as pn
import geoviews as gv
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import regrid, shade
from bokeh.io import output_file, save, show
from bokeh.models import HoverTool
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.plotting import figure

import envi_reader as es

In [2]:
nodata = 1


def read_tiff(path):
    return xr.open_rasterio(path).load()


def one_band(b):
    xs, ys = b['x'], b['y']
    print(xs, ys)
    b = ds.utils.orient_array(b)
    a = (np.where(np.logical_or(np.isnan(b), b <= nodata), 0, 255)).astype(np.uint8)
    col, rows = b.shape
    return hv.RGB((xs, ys[::-1], b, b, b, a), vdims=list('RGBA'))


def combine_bands(band):
    xs, ys = band['x'], band['y']
    r, g, b = [ds.utils.orient_array(img) for img in band]
    a = (np.where(np.logical_or(np.isnan(r), r <= nodata), 0, 255)).astype(np.uint8)
    return hv.RGB((xs, ys[::-1], r, g, b, a), vdims=list('RGBA'))


In [3]:
file_path = '/Users/jackson/Documents/code/bokeh/data/rgb20160212_003501_700591.tif'
band = read_tiff(file_path)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/rasterio/__init__.py:216: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, **kwargs)


In [6]:
band

<xarray.DataArray (band: 3, y: 1374, x: 1955)>
array([[[10295, 10289, 10285, ...,  7076,  6731,  6754],
        [ 9829,  9910,  9942, ...,  6759,  6803,  6601],
        [ 9624,  9735, 10086, ...,  6585,  6772,  6523],
        ...,
        [ 8977,  8723,  8867, ..., 10329, 10242,  9777],
        [ 8108,  8288,  8830, ...,  9937, 10121,  9957],
        [ 7839,  8618,  9633, ...,  9897, 10181, 10126]],

       [[13834, 13102, 11839, ...,  8041,  7862,  7638],
        [13259, 12658, 11074, ...,  7908,  7671,  7335],
        [12533, 12161, 10678, ...,  7823,  7660,  7154],
        ...,
        [10709, 10983, 11603, ..., 10579, 10569, 10482],
        [ 9171, 10546, 12396, ..., 10556, 10565, 10711],
        [ 9143, 11136, 13489, ..., 10578, 10495, 10669]],

       [[12063, 11699, 10732, ...,  8605,  8352,  7963],
        [11349, 10650,  9754, ...,  8591,  8400,  7972],
        [10020,  9064,  9132, ...,  7800,  7758,  7568],
        ...,
        [10165,  9927, 10266, ..., 11536, 10443,  9469],
        [ 9254, 10221, 11585, ..., 12000, 11304,  9893],
        [ 9554, 10857, 12205, ..., 11825, 11639, 10284]]], dtype=uint16)
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 0.5 1.5 2.5 3.5 ... 1.372e+03 1.372e+03 1.374e+03
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 1.952e+03 1.954e+03 1.954e+03
Attributes:
    transform:               (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
    res:                     (1.0, -1.0)
    is_tiled:                0
    nodatavals:              (nan, nan, nan)
    scales:                  (1.0, 1.0, 1.0)
    offsets:                 (0.0, 0.0, 0.0)
    TIFFTAG_RESOLUTIONUNIT:  2 (pixels/inch)
    TIFFTAG_XRESOLUTION:     72
    TIFFTAG_YRESOLUTION:     72

In [5]:
band['y'][::-1]

<xarray.DataArray 'y' (y: 1374)>
array([1.3735e+03, 1.3725e+03, 1.3715e+03, ..., 2.5000e+00, 1.5000e+00,
       5.0000e-01])
Coordinates:
  * y        (y) float64 1.374e+03 1.372e+03 1.372e+03 1.37e+03 ... 2.5 1.5 0.5